In [ ]:
import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import numpy as np
import theano
from sklearn import preprocessing

import plotly.graph_objects as go

# 1

> First, compute the entropy of each island’s birb distribution. Interpret these entropy values.

In [ ]:
data = [['Island 1', 0.2, 0.2, 0.2, 0.2, 0.2], ['Island 2', 0.8, 0.1, 0.05, 0.025, 0.025], ['Island 3', 0.05, 0.15, 0.7, 0.05, 0.05]] 

df = (pd.DataFrame(data, columns = ['Island', 'Bird A', 'Bird B', 'Bird C', 'Bird D', 'Bird E'])
      .set_index("Island")
     )

In [ ]:
entropy = df.apply(lambda x: stats.entropy(x.values), axis = 1)

In [ ]:
entropy

Entropy is a measure of uncertainty, with higher entropy meaning a more uniform distribution (and thus less surprisal at any given outcome). Island 1 has an equal distribution of all bird types (completely uniform), so its entropy is the largest of the three. Island 3 has mostly Bird A (80%), so its bird distribution has the lowest entropy. Island 2 is somewhere in between the other two.

> Second, use each island’s birb distribution to predict the other two. This means to compute the K-L Divergence of each island from the others, treating each island as if it were a statistical model of the other islands. You should end up with 6 different K-L Divergence values. Which island predicts the others best? Why?

In [ ]:
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

for i in range(df.shape[0]):
    df[f'KL_island{i+1}'] = df.apply(lambda x: kl_divergence(x.values, df.iloc[i].values), axis = 1)

In [ ]:
df

KL divergence offers a way to measure the surprisal of one distribution using another as a model. As we might expect, using Island 1 as a model results in, on average, lower divergence scores for the other islands. This is because Island 1 has maximum entropy - its uniform distribution of birds means it has no strong expectations that could be violated by the life on other islands. On the other hand, Islands 2 and 3 each have relatively peaked distributions, with just a single bird variety comprising the majority of life. As such, using them as a model means we expect that bird, and are subsequently very surprised when traveling to another island where that bird is much less prevalent.

# 2

> Recall the marriage, age, and happiness collider bias example from Chapter 6. Run models `m6.9` and `m6.10` again. Compare these two models using WAIC (or LOO, they will produce identical results). Which model is expected to make better predictions? Which model provides the correct causal inference about the influence of age on happiness? Can you explain why the answers to these two questions disagree?

First, pre-processing according to the models' specifications:

In [ ]:
happy = pd.read_csv('../../data/happiness.csv', header=0)
happy.head()

In [ ]:
adults = happy[happy['age'] > 17].copy()
adults['age'] = (adults['age'] - 18) / (65 - 18)

`m6.9`

In [ ]:
with pm.Model() as model69:
    # data
    happiness = adults['happiness']
    age = adults['age']
    married = adults['married'].values
    
    # priors
    alpha = pm.Normal('alpha', mu = 0, sigma = 1, shape = 2) # an intercept per married category
    beta_A = pm.Normal('beta_A', mu = 0, sigma = 2)
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha[married] + beta_A * age
    happiness_hat = pm.Normal('happiness_hat', mu = mu, sigma = sigma, observed = happiness)
    
    # sampling
    posterior69 = pm.sample(draws = 1000, tune = 1000)
    posterior_pred69 = pm.sample_posterior_predictive(posterior69)
    

In [ ]:
pm.summary(posterior69, alpha = .11).round(2)

Here, the model estimates a negative association between age and happiness, even though the data generating process assures that there is no such connection.

In [ ]:
with pm.Model() as model610:
    # data
    happiness = adults['happiness']
    age = adults['age']
    married = adults['married'].values
    
    # priors
    alpha = pm.Normal('alpha', mu = 0, sigma = 1) 
    beta_A = pm.Normal('beta_A', mu = 0, sigma = 2)
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha + beta_A * age
    happiness_hat = pm.Normal('happiness_hat', mu = mu, sigma = sigma, observed = happiness)
    
    # sampling
    posterior610 = pm.sample(draws = 1000, tune = 1000)
    posterior_pred610 = pm.sample_posterior_predictive(posterior610)
    

In [ ]:
pm.summary(posterior610, alpha = .11).round(2)

When we omit marriage as an indicator variable, the age effect goes away.

Now, let's compare these two models:

In [ ]:
model69.name = "model69"
model610.name = "model610"

In [ ]:
pm.compare({model69: posterior69, model610: posterior610})

`m6.9` has the lower WAIC value, even though it's confounded.

# 3

> Reconsider the urban fox analysis from last week’s homework. Use WAIC or LOO based model comparison on five different models, each using `weight` as the outcome, and containing these sets of predictor variables:

In [ ]:
foxes = pd.read_csv('../../data/foxes.csv', sep=';', header=0)
foxes[['avgfood','groupsize','area','weight']] = preprocessing.scale(foxes[['avgfood','groupsize','area','weight']])
foxes.head()

Since we'll be using the same data for all our models, we can set the feature vectors up at the start:

In [ ]:
avgfood = theano.shared(np.array(foxes.avgfood))
groupsize = theano.shared(np.array(foxes.groupsize))
area = theano.shared(np.array(foxes.area))
weight = theano.shared(np.array(foxes.weight))

In [ ]:
with pm.Model() as model1:
    #priors
    alpha = pm.Normal('alpha', 0, .5)
    beta_food = pm.Normal('beta_food', 0, .5)
    beta_groupsize = pm.Normal('beta_groupsize', 0, .5)
    beta_area = pm.Normal('beta_area', 0, .5)
    
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha + beta_food * avgfood + beta_groupsize * groupsize + beta_area* area
    weight_hat = pm.Normal('weight_hat', mu = mu, sigma = sigma, observed = weight)
    
    #sampling
    posterior1 = pm.sample(draws = 1000, tune = 1000)
    

In [ ]:
with pm.Model() as model2:
    #priors
    alpha = pm.Normal('alpha', 0, .5)
    beta_food = pm.Normal('beta_food', 0, .5)
    beta_groupsize = pm.Normal('beta_groupsize', 0, .5)
    
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha + beta_food * avgfood + beta_groupsize * groupsize 
    weight_hat = pm.Normal('weight_hat', mu = mu, sigma = sigma, observed = weight)
    
    #sampling
    posterior2 = pm.sample(draws = 1000, tune = 1000)
    

In [ ]:
with pm.Model() as model3:
    #priors
    alpha = pm.Normal('alpha', 0, .5)
    beta_area = pm.Normal('beta_area', 0, .5)
    beta_groupsize = pm.Normal('beta_groupsize', 0, .5)
    
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha + beta_area * area + beta_groupsize * groupsize 
    weight_hat = pm.Normal('weight_hat', mu = mu, sigma = sigma, observed = weight)
    
    #sampling
    posterior3 = pm.sample(draws = 1000, tune = 1000)
    

In [ ]:
with pm.Model() as model4:
    #priors
    alpha = pm.Normal('alpha', 0, .5)
    beta_food = pm.Normal('beta_food', 0, .5)    
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha + beta_food * avgfood
    weight_hat = pm.Normal('weight_hat', mu = mu, sigma = sigma, observed = weight)
    
    #sampling
    posterior4 = pm.sample(draws = 1000, tune = 1000)
    

In [ ]:
with pm.Model() as model5:
    #priors
    alpha = pm.Normal('alpha', 0, .5)
    beta_area = pm.Normal('beta_area', 0, .5)
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha + beta_area * area
    weight_hat = pm.Normal('weight_hat', mu = mu, sigma = sigma, observed = weight)
    
    #sampling
    posterior5 = pm.sample(draws = 1000, tune = 1000)
    

In [ ]:
model1.name = 'model1'
model2.name = 'model2'
model3.name = 'model3'
model4.name = 'model4'
model5.name = 'model5'

In [ ]:
pm.compare({
    model1: posterior1,
    model2: posterior2,
    model3: posterior3,
    model4: posterior4,
    model5: posterior5
    })